<a href="https://colab.research.google.com/github/Ashwinkarnati/MLProjects/blob/main/Fake_News_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

About the DataSet:
1. id: unique id for a news article
2. title: the title of a news article
3. author:author of the news article
4. text: the text of the article,could be incomplete
5. label: a label that marks whether the new is real or fake:
    1: Fake news
    0: Real news


In [ ]:
!pip install kaggle


In [1]:
from google.colab import files
files.upload()  # Upload kaggle.json


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"karnatiashwin","key":"e6c51fe0c68caecc4fc3c80bb369a044"}'}

In [2]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json  # Set permissions to read and write


In [3]:
!kaggle competitions download -c fake-news

 99% 46.0M/46.5M [00:01<00:00, 40.6MB/s]
100% 46.5M/46.5M [00:01<00:00, 27.6MB/s]


In [4]:
!kaggle competitions download -c fake-news

fake-news.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
!unzip fake-news.zip


Archive:  fake-news.zip
  inflating: submit.csv              
  inflating: test.csv                
  inflating: train.csv               


Importing thr Dependencies

In [113]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [114]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [115]:
#printing stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre-Processing

In [116]:
#loading the dataset to a pandas DataFrame
news_dataset=pd.read_csv('train.csv')
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [117]:
#counting the number of missing values in the dataset
news_dataset.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [118]:
news_dataset.shape


(20800, 5)

In [119]:
news_dataset=news_dataset.fillna('')

In [120]:
news_dataset.isnull().sum()

,0
id,0
title,0
author,0
text,0
label,0


In [121]:
#merginig the title and author name
news_dataset['content']=news_dataset['author']+" "+news_dataset['title']

In [122]:
news_dataset.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [123]:
x=news_dataset.drop(columns='label',axis=1)
y=news_dataset['label']
print(x,y,end="\n")

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

Stemming:
it is the process of producing the word to the root word

examples:
actor,acting,actress --> act

In [124]:
port_stem=PorterStemmer()

In [125]:
def stemming(content):
  if not isinstance(content, str):
        content = ""
  stemmed_content=re.sub('[^a-zA-Z]'," ",content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content=" ".join(stemmed_content)
  return stemmed_content

In [126]:
news_dataset['content']=news_dataset['content'].apply(stemming)

In [127]:
print(news_dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [128]:
#seperating the data and label
print(news_dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [129]:
x=news_dataset['content'].values
y=news_dataset['label'].values

In [130]:
print(x,y)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv'] [1 0 1 ... 0 1 1]


In [131]:
#converting the textual data to numerical data
vectorizer=TfidfVectorizer()
vectorizer.fit(x)
x=vectorizer.transform(x)

In [132]:
print(x)

  (0, 267)	0.2701012497770876
  (0, 2483)	0.36765196867972083
  (0, 2959)	0.24684501285337127
  (0, 3600)	0.3598939188262558
  (0, 3792)	0.27053324808454915
  (0, 4973)	0.23331696690935097
  (0, 7005)	0.2187416908935914
  (0, 7692)	0.24785219520671598
  (0, 8630)	0.2921251408704368
  (0, 8909)	0.36359638063260746
  (0, 13473)	0.2565896679337956
  (0, 15686)	0.2848506356272864
  (1, 1497)	0.2939891562094648
  (1, 1894)	0.15521974226349364
  (1, 2223)	0.3827320386859759
  (1, 2813)	0.19094574062359204
  (1, 3568)	0.26373768806048464
  (1, 5503)	0.7143299355715573
  (1, 6816)	0.1904660198296849
  (1, 16799)	0.30071745655510157
  (2, 2943)	0.3179886800654691
  (2, 3103)	0.46097489583229645
  (2, 5389)	0.3866530551182615
  (2, 5968)	0.3474613386728292
  (2, 9620)	0.49351492943649944
  :	:
  (20797, 3643)	0.2115550061362374
  (20797, 7042)	0.21799048897828685
  (20797, 8364)	0.22322585870464115
  (20797, 8988)	0.36160868928090795
  (20797, 9518)	0.29542040034203126
  (20797, 9588)	0.17455348

In [147]:
test_data = pd.read_csv('test.csv')
test_data.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [148]:
test_data.shape
test_data.isnull().sum()

,0
id,0
title,122
author,503
text,7


In [157]:
test_data.fillna("",inplace=True)
# Ensure 'author' and 'title' columns have no NaN values
test_data['author'] = test_data['author'].fillna("")
test_data['title'] = test_data['title'].fillna("")

test_data['content']=test_data['author']+" "+test_data['title']

In [158]:
df = pd.read_csv('submit.csv')

# Merge 'id' to get correct labels for y_test
merged_test = test_data[['id']].merge(df[['id', 'label']], on='id', how='left')
y_test = merged_test['label'].values
# Apply stemming function safely
if 'content' in test_data.columns:
    test_data['content'] = test_data['content'].astype(str).apply(stemming)
else:
    print("Error: 'content' column not found in test_data")

In [159]:
# Debugging: Show first few rows
print(test_data.head())
print(merged_test.head())

      id                                              title  \
0  20800  Specter of Trump Loosens Tongues, if Not Purse...   
1  20801  Russian warships ready to strike terrorists ne...   
2  20802  #NoDAPL: Native American Leaders Vow to Stay A...   
3  20803  Tim Tebow Will Attempt Another Comeback, This ...   
4  20804                    Keiser Report: Meme Wars (E995)   

                    author                                               text  \
0         David Streitfeld  PALO ALTO, Calif.  —   After years of scorning...   
1                           Russian warships ready to strike terrorists ne...   
2            Common Dreams  Videos #NoDAPL: Native American Leaders Vow to...   
3            Daniel Victor  If at first you don’t succeed, try a different...   
4  Truth Broadcast Network  42 mins ago 1 Views 0 Comments 0 Likes 'For th...   

                                             content  
0  david streitfeld specter trump loosen tongu pu...  
1  russian warship readi

In [160]:

print(test_data['content'])

0       david streitfeld specter trump loosen tongu pu...
1       russian warship readi strike terrorist near al...
2       common dream nodapl nativ american leader vow ...
3       daniel victor tim tebow attempt anoth comeback...
4        truth broadcast network keiser report meme war e
                              ...                        
5195    jodi rosen bangladeshi traffic jam never end n...
5196    sheryl gay stolberg john kasich sign one abort...
5197    mike mcphate california today exactli sushi ne...
5198                us marin deploy russian border norway
5199        teddi wayn awkward sex onscreen new york time
Name: content, Length: 5200, dtype: object


In [161]:
x_test=test_data['content'].values

In [162]:

x_test=vectorizer.transform(x_test)
print(x_test)

  (0, 3623)	0.209228842739656
  (0, 8842)	0.36912420870480717
  (0, 10306)	0.08568694729690247
  (0, 12030)	0.38547947461491155
  (0, 13850)	0.32956045112215354
  (0, 14591)	0.34467447913058513
  (0, 14605)	0.3970837204511862
  (0, 15295)	0.08697876504498342
  (0, 15368)	0.38547947461491155
  (0, 15582)	0.11032513315099028
  (0, 16088)	0.3205594987716609
  (0, 16996)	0.08864595251126647
  (1, 347)	0.3487744590431239
  (1, 10219)	0.37818137822242154
  (1, 12300)	0.3838422559012773
  (1, 13048)	0.3052969733971102
  (1, 14603)	0.35559908473568963
  (1, 15142)	0.3550835494981517
  (1, 16473)	0.4921981982038152
  (2, 469)	0.20423466307498248
  (2, 2981)	0.2974392647613127
  (2, 4337)	0.2945337589015841
  (2, 5352)	0.29644778940921024
  (2, 8508)	0.30056564685986675
  (2, 8520)	0.24040124223443918
  :	:
  (5196, 15295)	0.08170410847188171
  (5196, 16198)	0.3273837588421952
  (5196, 16996)	0.08327019262492417
  (5197, 2184)	0.3217750895378809
  (5197, 5005)	0.4638294166150902
  (5197, 9405)	0

In [163]:
model=LogisticRegression()


In [164]:
model.fit(x,y)

LogisticRegression()

Accuracy Score

In [165]:
#accuracy score on the training data
x_train_prediction=model.predict(x)
training_data_accuracy=accuracy_score(x_train_prediction,y)


In [166]:
print("Accuracy score of the training data: ",training_data_accuracy)


Accuracy score of the training data:  0.988125


In [167]:
#accuracy score on the test data
x_test_prediction=model.predict(x_test)
testing_data_accuracy=accuracy_score(x_test_prediction,y_test)

In [168]:
print("Accuracy score of the testing data : ",testing_data_accuracy)

Accuracy score of the testing data :  0.6411538461538462


Making a Predictive System

In [171]:
x_new = x_test[1]
prediction=model.predict(x_new)
print(prediction)
print(y_test[1])
if prediction==0:
  print("Real News")
else:
  print("Fake News")

[1]
1
Fake News


In [170]:
print(x_new)

  (0, 469)	0.20423466307498248
  (0, 2981)	0.2974392647613127
  (0, 4337)	0.2945337589015841
  (0, 5352)	0.29644778940921024
  (0, 8508)	0.30056564685986675
  (0, 8520)	0.24040124223443918
  (0, 10176)	0.3194727535249622
  (0, 10419)	0.3598282239050809
  (0, 11598)	0.21447077225895872
  (0, 14457)	0.3063702180182661
  (0, 16354)	0.2909537422615254
  (0, 16746)	0.30166289151474346
